# $\mbox{GOTO-Programme}$

Ein $\mbox{GOTO}$-Programm $P$ besteht aus einer geordneten Folge von
Anweisungen, ist also ein Vektor von Anweisungen. $P[i]$ bezeichnet
die $i$-te dieser Anweisungen. Dies ermöglicht bedingte und unbedingte
Sprünge zu den Anweisungen. Bei einem $\mbox{GOTO}$-Programm
$P=<A_1,A_2,\ldots,A_n>$ mit den Variablen $x_0,\ldots,x_m$ kann jede
Zeile eine der folgenden Anweisungen sein:
* $x_i:=x_j\pm c$ (Zuweisung, $0\leq i,j\leq m, c\in\mathbb{N}$)
* $\mbox{GOTO}\ k$ (unbedingter Sprung zu Anweisung $1\leq k\leq n$)
* $\mbox{IF}\ x_i=c\ \mbox{THEN}\ \mbox{GOTO}\ k$ (bedingter Sprung zu Anweisung)              $1\leq k\leq n$)
* $\mbox{HALT}$ (Programm-Ende)        

In [4]:
from pyparsing import Literal, Group, Forward, Word, alphas, nums, Combine, delimitedList, ZeroOrMore
ASSIGN         =  Literal(":=")
EQ             =  Literal("=")
ADD            =  Literal("+")
SUB            =  Literal("-")
IF             =  Literal("IF")
THEN           =  Literal("THEN")
GOTO           =  Literal("GOTO")
HALT           =  Literal("HALT")
VAR_PREFIX     =  Literal("x")

var            =  Combine(VAR_PREFIX+Word(nums))
number         =  Combine(Word(nums)|Word(nums))
add_assignment =  Group(var + ASSIGN + var + ADD + number)
sub_assignment =  Group(var + ASSIGN + var + SUB + number)
assignment     =  add_assignment | sub_assignment
goto_statement =  Group(GOTO + number)
if_statement   =  Group(IF + var + EQ + number + THEN + GOTO + number)
halt_statement =  Group(HALT)
statement      =  assignment | goto_statement | if_statement | halt_statement 
program        =  delimitedList(statement,delim=";")

## Tests für die Grammatik

In [5]:
for prog_str in ("x0:= x1 + 0",
                 "x0:= x1 + 0;IF x2=0 THEN GOTO 6;x2:=x2-1;x0:=x0+2;GOTO 2;HALT",
                ) :
    prog = program.parseString(prog_str, parseAll=True)
    print(prog)

[['x0', ':=', 'x1', '+', '0']]
[['x0', ':=', 'x1', '+', '0'], ['IF', 'x2', '=', '0', 'THEN', 'GOTO', '6'], ['x2', ':=', 'x2', '-', '1'], ['x0', ':=', 'x0', '+', '2'], ['GOTO', '2'], ['HALT']]


In [6]:
def isAddAssignment(statement) : return len(statement)>1 and statement[1]==":=" and statement[3]=="+"
def isSubAssignment(statement) : return len(statement)>1 and statement[1]==":=" and statement[3]=="-"
def isGoto(statement) : return len(statement)>0 and statement[0]=="GOTO"
def isConditionalGoto(statement) : return len(statement)>0 and statement[0]=="IF"
def isHalt(statement) : return len(statement)>0 and statement[0]=="HALT"

In [8]:
def interpretGoto(p, b) :
    pc        = b['pc']
    while True :
        s = p[pc]
        if isHalt(s) : break
        elif isAddAssignment(s) :
            b[s[0]] = b[s[2]] + int(s[4])
            b['pc']=pc+1
        elif isSubAssignment(s) :
            x = b[s[2]] - int(s[4])
            b[s[0]] = x if x>0 else 0
            b['pc']=pc+1
        elif isGoto(s) :
            b['pc']=int(s[1])
        elif isConditionalGoto(s) :
            if b[s[1]]==int(s[3]) : b['pc']=int(s[6])
            else :          b['pc']=pc+1
        else : 
            print("Error in Program! Statement: "+str(s)+"(maybe macro still present?)")
            break
        pc=b['pc']
    return b

def semanticsGoto(p) :
    def semantics(b) : return interpretGoto(p,b)
    return semantics




In [9]:
prog_str = "x0:= x1 + 0;IF x2=0 THEN GOTO 5;x2:=x2-1;x0:=x0+2;GOTO 1;HALT"
prog = program.parseString(prog_str, parseAll=True)
print(prog)
sem=semanticsGoto(prog)

args = {'pc': 0, 'x0':0,'x1':3,'x2':30}
res=sem(args)
print(res)

[['x0', ':=', 'x1', '+', '0'], ['IF', 'x2', '=', '0', 'THEN', 'GOTO', '5'], ['x2', ':=', 'x2', '-', '1'], ['x0', ':=', 'x0', '+', '2'], ['GOTO', '1'], ['HALT']]
{'pc': 5, 'x0': 63, 'x1': 3, 'x2': 0}


## Macro call syntax

In [11]:
LBRACKET       =  Literal("(")
RBRACKET       =  Literal(")").suppress()
macro_name     =  Combine(Word(alphas))
macro_statement=  Group(macro_name + LBRACKET + delimitedList(var,delim=",")+RBRACKET)
statement      =  assignment | goto_statement | if_statement | halt_statement | macro_statement
program        =  delimitedList(statement,delim=";")

def isMacro(statement) : return len(statement)>0 and statement[1]=="("

In [12]:
for prog_str in (
                 "x0:= x1 + 0;IF x2=0 THEN GOTO 6;x2:=x2-1;add(x0,x1,x2);GOTO 2;HALT",
                ) :
    prog = program.parseString(prog_str, parseAll=True)
    print(prog)

[['x0', ':=', 'x1', '+', '0'], ['IF', 'x2', '=', '0', 'THEN', 'GOTO', '6'], ['x2', ':=', 'x2', '-', '1'], ['add', '(', 'x0', 'x1', 'x2'], ['GOTO', '2'], ['HALT']]


## Macros definition
We define macros as triples consisting of
* name
* number of arguments
* code

argument values can be used in code by \n notation (e.g. \2 is the second argument)</br>
The following example shows a macro that implements addition of two variables. x100 is used as a local/helper variable.

In [13]:
add_macro = ("add",3,r"x100:=\3+0; \1:= \2 + 0;IF x100=0 THEN GOTO 6;x100:=x100-1;\1:=\1+1;GOTO 1;HALT")

### Application of macros
Applying macros to a program is a multi-step process

1. find occurence of macro in code, store line $l$ and call $c$ and replace call by $GOTO\ N$ where N is the number of lines in the program (macro will be appended to the program)
2. replace arguments in macro according to $c$
4. add $N$ to all $GOTO$ and conditional Goto statements in macro
3. replace $HALT$ statements with $GOTO\ l$
5. append macro to program

In [14]:
import re

def apply_macro(m,p) :
    def macro_call_to_str(m) : return m[0] +"("+",".join(m[2:])+")"
    def string_to_parse_cell(s) : return program.parseString(s)[0]
    name = m[0]
    pattern = m[0] + "\\(" + "(x\\d*),"*(m[1]-1) + "(x\\d*)\\)"
    code = m[2]
    N = len(p)
    for i in range(N) :
        s = p[i]
        last = len(p) # might not be N any more because of macro appended
        if s[0]==name :
            p[i] = string_to_parse_cell("GOTO "+str(last))  # create a *statement* to jump to macro
            newcode=re.sub(pattern,code,macro_call_to_str(s)) # replace arg-placeholders
            macro_instance=program.parseString(newcode,parseAll=True)
            for s in macro_instance :
                if   isGoto(s) : s[1] = str(int(s[1])+last)
                elif isConditionalGoto(s) : s[6] = str(int(s[6])+last)
                elif isHalt(s) : 
                    s[0]="GOTO" 
                    s.append(str(i+1))
            p=p+macro_instance
    return p


In [15]:
#prog_str = "x0:= x1 + 0;IF x2=0 THEN GOTO 6;x2:=x2-1;add(x0,x0,x2);add(x0,x0,x2);GOTO 1;HALT"
prog_str = "x0:= x1 + 0;IF x2=0 THEN GOTO 5;x2:=x2-1;add(x0,x0,x2);GOTO 1;HALT"
prog=program.parseString(prog_str, parseAll=True)
prog=apply_macro(add_macro,prog)
sem=semanticsGoto(prog)
print(prog)

args = {'pc': 0, 'x0':0,'x1':3,'x2':10}
res=sem(args)
print(res)

[['x0', ':=', 'x1', '+', '0'], ['IF', 'x2', '=', '0', 'THEN', 'GOTO', '5'], ['x2', ':=', 'x2', '-', '1'], ['GOTO', '6'], ['GOTO', '1'], ['HALT'], ['x100', ':=', 'x2', '+', '0'], ['x0', ':=', 'x0', '+', '0'], ['IF', 'x100', '=', '0', 'THEN', 'GOTO', '12'], ['x100', ':=', 'x100', '-', '1'], ['x0', ':=', 'x0', '+', '1'], ['GOTO', '7'], ['GOTO', '4']]
{'pc': 5, 'x0': 48, 'x1': 3, 'x2': 0, 'x100': 0}


In [29]:
def f(*prog_and_macros):
    P=prog_and_macros[0]
    tree=program.parseString(P, parseAll=True)
    for m in prog_and_macros[1:]:
        tree=apply_macro(m,tree)
    sem =semanticsGoto(tree)

    def fP(*args):
        args = [0]+list(args)
        beta={'x'+str(i): args[i] for i in range(len(args))}
        beta['pc']=0
        return sem(beta)['x0']
    
    return fP

In [32]:
fP = f(prog_str, add_macro)
fP(3,10)

48